<a href="https://colab.research.google.com/github/timovijn/ElectricityPriceForecasting/blob/master/LSTM/LSTM%20confidence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bayesian LSTM for electricity price prediction**

**Monte Carlo dropout as an approximation for Bayesian Inference**

Timo Vijn

# Required packages

In [22]:
import pandas as pd
import numpy as np

hex_salmon = '#F68F83'
hex_gold = '#BC9661'
hex_indigo = '#2D2E5F'
hex_maroon = '#8C4750'
hex_white = '#FAFAFA'
hex_blue = '#7EB5D2'

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates

import matplotlib.font_manager as font_manager
mpl.font_manager._rebuild()

mpl.rcParams['font.family'] = 'SF Mono'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.titleweight'] = 'semibold'
mpl.rcParams['axes.labelweight'] = 'medium'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[hex_indigo, hex_salmon, hex_maroon])
mpl.rcParams["figure.titlesize"] = 'large'
mpl.rcParams["figure.titleweight"] = 'semibold'

from termcolor import colored

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, accuracy_score

import tensorflow as tf

# Load data

## Clone from Github

In [2]:
! pip install ghclone &> /dev/null

! ghclone https://github.com/timovijn/ElectricityPriceForecasting/tree/master/LSTM

Cloning into 'LSTM'...
done.


## Load features dataframe

In [3]:
features = pd.read_pickle(f"./LSTM/features2.pkl")

print(colored(f'Load complete', 'green'))

Load complete


In [4]:
features.head(5)

,ID3,VOL,MCP,LOAD,LOAD_F,LOAD_FE,ID3 (-4),ID3 (-5),ID3 (-6),ID3 (-7),ID3 (-8),ID3 (-9),ID3 (-10),ID3 (-11),ID3 (-12),ID3 (-13),ID3 (-14),ID3 (-15),ID3 (-16),ID3 (-17),ID3 (-18),ID3 (-19),ID3 (-20),ID3 (-21),ID3 (-22),ID3 (-23),ID3 (-24),ID3 (-25),ID3 (-26),ID3 (-27),ID3 (-28),ID3 (-29),ID3 (-30),ID3 (-31),ID3 (-32),ID3 (-33),ID3 (-34),ID3 (-35),ID3 (-36),ID3 (-37),...,VOL (-22),VOL (-23),VOL (-24),Gen. forecast,Gen. (s) forecast,Gen. (w_off) forecast,Gen. (w_on) forecast,DOW,DOW 0,DOW 1,DOW 2,DOW 3,DOW 4,DOW 5,DOW 6,HOD,HOD 0,HOD 1,HOD 2,HOD 3,HOD 4,HOD 5,HOD 6,HOD 7,HOD 8,HOD 9,HOD 10,HOD 11,HOD 12,HOD 13,HOD 14,HOD 15,HOD 16,HOD 17,HOD 18,HOD 19,HOD 20,HOD 21,HOD 22,HOD 23
2015-01-01 05:00:00+00:00,27.500000,29.0,27.41,8291.75,9994.25,1702.50,25.625000,29.100000,26.357143,26.073529,27.538462,31.480519,36.581111,35.371102,35.561247,26.881944,34.906250,36.000000,23.372727,23.372727,29.951386,29.862160,28.795380,23.342037,28.928153,39.433384,37.986468,27.000000,41.867257,51.000000,51.590164,51.900000,41.067436,40.806625,40.597342,43.215488,41.725543,53.263736,54.308642,59.623967,...,231.0,624.0,669.3,11869.0,0.0,57.50,515.50,3,0,0,0,1,0,0,0,5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2015-01-01 06:00:00+00:00,26.780822,28.0,27.52,8638.25,11882.00,3243.75,29.100000,26.357143,26.073529,27.538462,31.480519,36.581111,35.371102,35.561247,26.881944,34.906250,36.000000,23.372727,23.372727,29.951386,29.862160,28.795380,23.342037,28.928153,39.433384,37.986468,27.000000,41.867257,51.000000,51.590164,51.900000,41.067436,40.806625,40.597342,43.215488,41.725543,53.263736,54.308642,59.623967,53.237458,...,624.0,669.3,722.9,14495.0,0.0,61.50,551.00,3,0,0,0,1,0,0,0,6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2015-01-01 07:00:00+00:00,25.852273,65.0,26.80,8927.25,13240.00,4312.75,26.357143,26.073529,27.538462,31.480519,36.581111,35.371102,35.561247,26.881944,34.906250,36.000000,23.372727,23.372727,29.951386,29.862160,28.795380,23.342037,28.928153,39.433384,37.986468,27.000000,41.867257,51.000000,51.590164,51.900000,41.067436,40.806625,40.597342,43.215488,41.725543,53.263736,54.308642,59.623967,53.237458,55.302469,...,669.3,722.9,655.2,16740.0,0.0,65.75,592.25,3,0,0,0,1,0,0,0,7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2015-01-01 08:00:00+00:00,24.400000,18.0,28.48,9312.50,13817.25,4504.75,26.073529,27.538462,31.480519,36.581111,35.371102,35.561247,26.881944,34.906250,36.000000,23.372727,23.372727,29.951386,29.862160,28.795380,23.342037,28.928153,39.433384,37.986468,27.000000,41.867257,51.000000,51.590164,51.900000,41.067436,40.806625,40.597342,43.215488,41.725543,53.263736,54.308642,59.623967,53.237458,55.302469,55.146552,...,722.9,655.2,0.0,16907.0,31.0,70.50,635.50,3,0,0,0,1,0,0,0,8,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2015-01-01 09:00:00+00:00,25.625000,15.0,27.71,9933.25,14072.50,4139.25,27.538462,31.480519,36.581111,35.371102,35.561247,26.881944,34.906250,36.000000,23.372727,23.372727,29.951386,29.862160,28.795380,23.342037,28.928153,39.433384,37.986468,27.000000,41.867257,51.000000,51.590164,51.900000,41.067436,40.806625,40.597342,43.215488,41.725543,53.263736,54.308642,59.623967,53.237458,55.302469,55.146552,54.536232,...,655.2,0.0,21.0,17313.0,97.0,72.75,654.00,3,0,0,0,1,0,0,0,9,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Extract features

In [5]:
features = features[['DOW', 'HOD', 'ID3']]

features = features.reset_index()

features = features.rename(columns={'index':'Date'})

In [6]:
features.head(5)

,Date,DOW,HOD,ID3
0,2015-01-01 05:00:00+00:00,3,5,27.500000
1,2015-01-01 06:00:00+00:00,3,6,26.780822
2,2015-01-01 07:00:00+00:00,3,7,25.852273
3,2015-01-01 08:00:00+00:00,3,8,24.400000
4,2015-01-01 09:00:00+00:00,3,9,25.625000


In [7]:
features.tail(5)

,Date,DOW,HOD,ID3
33840,2018-12-21 23:00:00+00:00,4,23,50.911996
33841,2018-12-22 00:00:00+00:00,5,0,53.554378
33842,2018-12-22 01:00:00+00:00,5,1,53.716854
33843,2018-12-22 02:00:00+00:00,5,2,52.950763
33844,2018-12-22 03:00:00+00:00,5,3,53.277543


In [8]:
import numpy as np

resample_df = features

datetime_columns = ['Date', 'DOW', 'HOD']
target_column = 'ID3'

feature_columns = datetime_columns + ['ID3']

resample_df = resample_df[feature_columns]

## Plot ID3

In [24]:
import plotly.express as px

plot_length = 24*7 - 10
plot_df = resample_df.copy(deep=True).iloc[:plot_length]
# plot_df['weekday'] = plot_df['date'].dt.weekday_name
plot_df['DOW'] = plot_df['Date'].dt.day_name()

fig = px.line(plot_df,
              x="Date",
              y="ID3", 
              color="DOW", 
              title="Log of Appliance Energy Consumption vs Time")

fig.layout.font.family = 'SF Mono'

fig.show()

# Prepare Training Data

For this example, we will use sliding windows of 10 points per each window (equivalent to 10 hours) to predict each next point. The window size can be altered via the `sequence_length` variable.

Min-Max scaling has also been fitted to the training data to aid the convergence of the neural network. 

In [10]:
from sklearn.preprocessing import MinMaxScaler

def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)

train_split = 0.7
n_train = int(train_split * len(resample_df))
n_test = len(resample_df) - n_train

# features = ['DOW', 'HOD', 'ID3']
features = ['ID3']
feature_array = resample_df[features].values

# Fit Scaler only on Training features
feature_scaler = MinMaxScaler()
feature_scaler.fit(feature_array[:n_train])
# Fit Scaler only on Training target values
target_scaler = MinMaxScaler()
target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

# Transfom on both Training and Test data
scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                            columns=features)

sequence_length = 10
X, y = create_sliding_window(scaled_array, 
                             sequence_length)

X_train = X[:n_train]
y_train = y[:n_train]

X_test = X[n_train:]
y_test = y[n_train:]

In [11]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
# import numpy as np

# inputs = features[['ID3']]
# outputs = features[['ID3']]

# inputs_train = inputs[inputs.index.year < 2018]
# outputs_train = outputs[outputs.index.year < 2018]

# inputs_test = inputs[inputs.index.year == 2018]
# outputs_test = outputs[outputs.index.year == 2018]

# # Scale inputs
# scaler = MinMaxScaler()

# inputs_train = pd.DataFrame(scaler.fit_transform(inputs_train), columns = inputs_train.columns)
# inputs_test = pd.DataFrame(scaler.transform(inputs_test), columns = inputs_test.columns)

# time_steps = 6
# for_periods = 1
# lag = 4                           

# # X_train and y_train

# X_train = []
# y_train = []

# for i in range(0, len(outputs_train)-time_steps-lag+1):
#     for ii in range(0, time_steps):
#         X_train.extend(inputs_train.iloc[i+ii].to_numpy())
#     y_train.extend(outputs_train.iloc[i+time_steps+lag-1].to_numpy())

# X_train, y_train = np.array(X_train), np.array(y_train)

# X_train = X_train.reshape(y_train.shape[0], time_steps, inputs.shape[1])

# # X_test and y_test

# X_test = []
# y_test = []

# for i in range(0, len(outputs_test)-time_steps-lag+1):
#     for ii in range(0, time_steps):
#         X_test.extend(inputs_test.iloc[i+ii].to_numpy())
#     y_test.extend(outputs_test.iloc[i+time_steps+lag-1].to_numpy())

# X_test, y_test = np.array(X_test), np.array(y_test)

# X_test = X_test.reshape(y_test.shape[0], time_steps, inputs.shape[1])

# Define Bayesian LSTM Architecture

To demonstrate a simple working example of the Bayesian LSTM, the model as defined in Uber's paper has been used a starting point. The network architecture is as follows:

Encoder-Decoder Stage:
 - 1 uni-directional LSTM layer with 128 hidden units acts as an encoding layer to construct a fixed-dimension embedding state
 - 1 uni-directional LSTM layer with 32 hidden units acts as a decoding layer to produce predictions at future steps
 - Dropout is applied at **both** training and inference for both LSTM layers


 Predictor Stage:
 - 1 fully-connected output layer with 1 output (for predicting the target value) to produce a single value for the target variable


By allowing dropout at both training and testing time, the model simulates random sampling, thus allowing varying predictions that can be used to estimate the underlying distribution of the target value, enabling explicit model uncertainties.


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length):

        super(BayesianLSTM, self).__init__()

        self.hidden_size_1 = 128
        self.hidden_size_2 = 32
        self.n_layers = 1 # number of (stacked) LSTM layers

        self.lstm1 = nn.LSTM(n_features, 
                             self.hidden_size_1, 
                             num_layers=1,
                             batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=1,
                             batch_first=True)
        
        self.dense = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)
        output, _ = self.lstm1(x, hidden)
        output = F.dropout(output, p=0.5, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=0.5, training=True)
        output = self.dense(state[0].squeeze(0))
        
        return output
        
    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state
    
    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state
    
    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

### Begin Training

To train the Bayesian LSTM, we use the ADAM optimizer along with mini-batch gradient descent (`batch_size = 128`). For quick demonstration purposes, the model is trained for 150 epochs.

The Bayesian LSTM is trained on the first 70% of data points, using the aforementioned sliding windows of size 10. The remaining 30% of the dataset is held out purely for testing.

In [13]:
n_features = scaled_array.shape[-1]
# n_features = inputs.shape[1]
sequence_length = 10
output_length = 1

bayesian_lstm = BayesianLSTM(n_features=n_features,
                             output_length=output_length)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=0.01)

batch_size = 128
n_epochs = 5

In [14]:
bayesian_lstm.train()

for e in range(1, n_epochs+1):
    for b in range(0,len(X_train),batch_size):
        features = X_train[b:b+batch_size,:,:]
        target = y_train[b:b+batch_size]    

        X_batch = torch.tensor(features,dtype=torch.float32)    
        y_batch = torch.tensor(target,dtype=torch.float32)

        output = bayesian_lstm(X_batch) 
        loss = criterion(output.view(-1), y_batch)  

        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 

    if e % 1 == 0:
      print('epoch: ', e, 'loss: ', loss.item())

epoch:  1 loss:  0.002094875555485487
epoch:  2 loss:  0.0025086624082177877
epoch:  3 loss:  0.0013743899762630463
epoch:  4 loss:  0.0005374590982683003
epoch:  5 loss:  0.0005019349046051502


# Evaluating Model Performance

The Bayesian LSTM implemented is shown to produce reasonably accurate and sensible results on both the training and test sets, often comparable to other existing frequentist machine learning and deep learning methods.



In [15]:
offset = sequence_length

def inverse_transform(y):
  return target_scaler.inverse_transform(y.reshape(-1, 1))

training_df = pd.DataFrame()
training_df['Date'] = resample_df['Date'].iloc[offset:n_train + offset:1] 
training_predictions = bayesian_lstm.predict(X_train)
training_df['ID3'] = inverse_transform(training_predictions)
training_df['Source'] = 'Training Prediction'

training_truth_df = pd.DataFrame()
training_truth_df['Date'] = training_df['Date']
training_truth_df['ID3'] = resample_df['ID3'].iloc[offset:n_train + offset:1] 
training_truth_df['Source'] = 'True Values'

testing_df = pd.DataFrame()
testing_df['Date'] = resample_df['Date'].iloc[n_train + offset::1] 
testing_predictions = bayesian_lstm.predict(X_test)
testing_df['ID3'] = inverse_transform(testing_predictions)
testing_df['Source'] = 'Test Prediction'

testing_truth_df = pd.DataFrame()
testing_truth_df['Date'] = testing_df['Date']
testing_truth_df['ID3'] = resample_df['ID3'].iloc[n_train + offset::1] 
testing_truth_df['Source'] = 'True Values'

evaluation = pd.concat([training_df, 
                        testing_df,
                        training_truth_df,
                        testing_truth_df
                        ], axis=0)

In [16]:
fig = px.line(evaluation.loc[evaluation['Date'].between('2015-01-01', '2018-12-01')],
                 x="Date",
                 y="ID3",
                 color="Source",
                 title="Log of Appliance Energy Consumption in Wh vs Time")
fig.show()

# Uncertainty Quantification

The fact that stochastic dropouts are applied after each LSTM layer in the Bayesian LSTM enables users to interpret the model outputs as random samples from the posterior distribution of the target variable. 

This implies that by running multiple experiments/predictions, can approximate  parameters of the posterioir distribution, namely the mean and the variance, in order to create confidence intervals for each prediction.

In this example, we construct 99% confidence intervals that are three standard deviations away from the approximate mean of each prediction.

In [17]:
n_experiments = 100

test_uncertainty_df = pd.DataFrame()
test_uncertainty_df['Date'] = testing_df['Date']

for i in range(n_experiments):
  experiment_predictions = bayesian_lstm.predict(X_test)
  test_uncertainty_df['ID3_{}'.format(i)] = inverse_transform(experiment_predictions)

log_energy_consumption_df = test_uncertainty_df.filter(like='ID3', axis=1)
test_uncertainty_df['ID3_mean'] = log_energy_consumption_df.mean(axis=1)
test_uncertainty_df['ID3_std'] = log_energy_consumption_df.std(axis=1)

test_uncertainty_df = test_uncertainty_df[['Date', 'ID3_mean', 'ID3_std']]

In [18]:
test_uncertainty_df['lower_bound'] = test_uncertainty_df['ID3_mean'] - 3*test_uncertainty_df['ID3_std']
test_uncertainty_df['upper_bound'] = test_uncertainty_df['ID3_mean'] + 3*test_uncertainty_df['ID3_std']

In [19]:
import plotly.graph_objects as go

test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['Date'].between('2018-10-01', '2018-10-04')]
truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['Date'].between('2018-10-01', '2018-10-04')]

upper_trace = go.Scatter(
    x=test_uncertainty_plot_df['Date'],
    y=test_uncertainty_plot_df['upper_bound'],
    mode='lines',
    fill=None,
    name='99% Upper Confidence Bound'
    )
lower_trace = go.Scatter(
    x=test_uncertainty_plot_df['Date'],
    y=test_uncertainty_plot_df['lower_bound'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 211, 0, 0.5)',
    name='99% Lower Confidence Bound'
    )
real_trace = go.Scatter(
    x=truth_uncertainty_plot_df['Date'],
    y=truth_uncertainty_plot_df['ID3'],
    mode='lines',
    fill=None,
    name='Real Values'
    )

data = [upper_trace, lower_trace, real_trace]

fig = go.Figure(data=data)
fig.update_layout(title='Uncertainty Quantification for Energy Consumption Test Data',
                   xaxis_title='Time',
                   yaxis_title='log_energy_consumption (log Wh)')

fig.layout.font.family = 'SF Mono'

fig.show()

#### Evaluating Uncertainty

Using multiple experiments above, 99% confidence intervals have been constructed for each the prediction of the target variable (the logarithm of appliance power consumption). While we can visually observe that the model is generally capturing the behavior of the time-series, approximately only 50% of the real data points lie within a 99% confidence interval from the mean prediction value.

Despite the relatively low percentage of points within the confidence interval, it must be noted that Bayesian Neural Networks only seek to quantify the epistemic model uncertainty and does not account for aleatoric uncertainty (i.e. noise).

In [20]:
bounds_df = pd.DataFrame()

# Using 99% confidence bounds
bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
bounds_df['prediction'] = test_uncertainty_plot_df['ID3_mean']
bounds_df['real_value'] = truth_uncertainty_plot_df['ID3']
bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                          (bounds_df['real_value'] <= bounds_df['upper_bound']))

print("Proportion of points contained within 99% confidence interval:", 
      bounds_df['contained'].mean())

Proportion of points contained within 99% confidence interval: 0.6712328767123288


# Conclusions

- Bayesian LSTMs have been able to produce comparable performance to their frequentist counterparts (all else being equal)
- Stochastic dropout enables users to approximate the posterior distribution of the target variable, \
and thus construct confidence intervals for each prediction 
- Bayesian Neural Networks only attempt to account for epistemic model uncertainty and do not necessarily address aleatoric uncertainty
- Computational overhead for repeated/multiple Bayesian LSTM predictions at inference to construct confidence intervals represent a potential challenge for real-time inference use-cases.